# **검색량과 데이터 테이블 합치기**

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
import time 
import sys 
import warnings 

rc = {
    "axes.facecolor": "#FFFFFF",       
    "figure.facecolor": "#FFFFFF",
    "axes.edgecolor": "#000000",
    "grid.color": "#CCCCCC",
    "font.family": "malgun gothic",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000",
    "ytick.color": "#000000",
    "grid.alpha": 0.4,
}

sns.set(rc = rc)
plt.rc('axes', unicode_minus = False)
warnings.filterwarnings(action = 'ignore')

In [2]:
df = pd.read_parquet("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/전체공연_원가격원등급추정결과.parquet")

# 3가지 키워드 검색량 데이터 추가 

# 예술의전당 콘서트홀 키워드
# 예술의전당 클래식 키워드
# 서울클래식 키워드 

concerthall_keyword = pd.read_excel("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/키워드사운드_예술의전당 콘서트홀_검색량.xlsx")
seoulartcenter_keyword = pd.read_excel("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/키워드사운드_예술의전당 클래식_검색량.xlsx")
seoulclassic_keyword = pd.read_excel("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/키워드사운드_서울 클래식_검색량.xlsx")

In [3]:
display(concerthall_keyword.head())

,날짜,키워드,PC 검색량,모바일 검색량,총 검색량
0,2016-01-01,예술의전당 콘서트홀,10,30,40
1,2016-01-02,예술의전당 콘서트홀,20,40,60
2,2016-01-03,예술의전당 콘서트홀,30,30,60
3,2016-01-04,예술의전당 콘서트홀,60,40,100
4,2016-01-05,예술의전당 콘서트홀,60,40,100


##### **공연 최초 예매일 전 일주일 간의 누적 검색량을 검색량으로 생각하도록 하자**

In [4]:
pre_open_date = df.loc[~df['pre_open_date'].isna(),'pre_open_date'].unique().tolist()
open_date = df.loc[df['pre_open_date'].isna(), 'open_date'].unique().tolist()

total_open = pre_open_date + open_date

In [5]:
print('유니크한 전체 공연 시간 :',df['전체공연시간'].nunique())
print('유니크한 전체 오픈 시간 : ', len(total_open))

유니크한 전체 공연 시간 : 151
유니크한 전체 오픈 시간 :  140


##### **공연 별로 중복된 티켓 예매일이 존재한다**

In [6]:
display(df['pre_open_date'].value_counts() / 2505)
display(df['open_date'].value_counts() / 2505)

2018-10-28    4.0
2018-12-10    4.0
2021-07-02    3.0
2022-07-30    3.0
2023-02-25    2.0
             ... 
2021-04-16    1.0
2021-04-24    1.0
2021-06-25    1.0
2021-06-04    1.0
2023-03-25    1.0
Name: pre_open_date, Length: 72, dtype: float64

2018-10-29    8.0
2018-12-30    5.0
2018-12-22    4.0
2018-12-10    4.0
2022-07-31    3.0
             ... 
2020-10-25    1.0
2020-09-20    1.0
2020-10-02    1.0
2020-08-24    1.0
2023-03-26    1.0
Name: open_date, Length: 132, dtype: float64

# **각 공연 별로 첫 예매 시작 일주일 전의 누적 검색량을 데이터프레임에 병합하기**

In [7]:
def keyword_calculator(data, concerthall, artcenter_classic,seoul_classic , days):
    
    data = data.copy()
    data['누적검색량'] = 0 # 맨 처음 시리즈를 모두 0으로 초기화 
    
    unique_date_list = data['전체공연시간'].unique()
    
    for unq_date in unique_date_list:
        cond = data['전체공연시간'] == unq_date
        
        if np.sum(data.loc[cond,'pre_open_date'].isna()): # 만약 선예매가 열린 적 없다면
            
            base_days = data.loc[cond, 'open_date'].unique()[0]
        else: # 만약 선예매가 열린 적이 있다면 
            base_days = data.loc[cond, 'pre_open_date'].unique()[0]
            
        end_days = pd.to_datetime(base_days) # YYYY-MM-DD 형태가 검색의 end 부분 
            
        start_days = end_days - pd.Timedelta(days = days)
            
        keyword_cond = (concerthall['날짜'] >= str(start_days)[:10]) & (concerthall['날짜'] <= str(end_days)[:10])
        
        concert_cumul_keyword = np.sum(concerthall.loc[keyword_cond, '총 검색량'])
        artcent_cumul_keyword = np.sum(artcenter_classic.loc[keyword_cond, '총 검색량'])
        seo_classic_cumul_keyword = np.sum(seoul_classic.loc[keyword_cond, '총 검색량'])
        
        data.loc[cond, '콘서트홀클래식_누적검색량'] = concert_cumul_keyword
        data.loc[cond, '예술의전당클래식_누적검색량'] = artcent_cumul_keyword
        data.loc[cond, '서울클래식_누적검색량'] = seo_classic_cumul_keyword
        
    return data
    

In [8]:
result = keyword_calculator(data = df,
                            concerthall = concerthall_keyword,
                            artcenter_classic = seoulartcenter_keyword,
                            seoul_classic = seoulclassic_keyword,
                            days = 7)

In [9]:
display(result.sample(5))
print(result.shape)

,seat,층,블록,열,넘버,X,Y,Z,대칭점,좌우시야각,...,전체거래시간,할인율,할인전가격,원가격추정,z_score,원등급추정,누적검색량,콘서트홀클래식_누적검색량,예술의전당클래식_누적검색량,서울클래식_누적검색량
76739,2층 B블록2열 1,2층,B블록,2,1,889,2377,675,2층 D블록2열 10,20.505839,...,2019-06-22 16:13:00,0.0,0,230000,1.00,5,0,1250.0,0.0,20.0
124347,2층 C블록2열 4,2층,C블록,2,4,99,2423,675,2층 C블록2열 8,2.339715,...,2019-10-24 09:12:00,0.0,0,25000,0.85,3,0,1310.0,0.0,70.0
147071,2층 A블록5열 3,2층,A블록,5,3,1825,1913,802,2층 E블록5열 18,43.651392,...,NaT,0.0,0,35000,0.16,2,0,720.0,0.0,70.0
90033,3층 C블록6열 12,3층,C블록,6,12,431,3044,1388,3층 E블록6열 1,8.058941,...,2019-07-07 09:27:00,0.0,45000,45000,0.00,2,0,1790.0,20.0,30.0
123946,1층 B블록22열 3,1층,B블록,22,3,1192,2667,332,1층 D블록22열 12,24.081975,...,2019-10-24 09:11:00,0.0,0,0,1.93,1,0,1310.0,0.0,70.0


(415830, 45)


In [10]:
result[['콘서트홀클래식_누적검색량', '예술의전당클래식_누적검색량', '서울클래식_누적검색량']].describe()

,콘서트홀클래식_누적검색량,예술의전당클래식_누적검색량,서울클래식_누적검색량
count,415830.000000,415830.000000,415830.000000
mean,908.072289,10.120482,51.084337
std,375.223944,17.834196,34.309478
min,110.000000,0.000000,0.000000
25%,610.000000,0.000000,20.000000
50%,935.000000,0.000000,45.000000
75%,1180.000000,20.000000,70.000000
max,2000.000000,80.000000,170.000000


In [11]:
result['누적검색량'] = result['콘서트홀클래식_누적검색량'] +  result['예술의전당클래식_누적검색량'] + result['서울클래식_누적검색량']

**3가지 검색 키워드 중 예술의 전당 콘서트홀 검색 키워드가 가장 많은 것으로 나타났다**

In [12]:
file_path = 'C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/'

result.to_parquet(file_path + '키워드_검색량_추가_클래식_데이터.parquet',index = False)